In [1]:
import requests
from datetime import datetime
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
MONTHS = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06",
    "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}

In [3]:
def get_url(start, stop, step):
    return "https://ssd.jpl.nasa.gov/api/horizons.api?" + "&".join(f"{k}={v}" for k, v in {
        "EPHEM_TYPE": "'VECTORS'",
        "COMMAND": "'399'",
        "CENTER": "'500@10'",
        "START_TIME": f"'{start}'",
        "STOP_TIME": f"'{stop}'",
        "STEP_SIZE": f"'{step}'",
        "OBJ_DATA": "'NO'",
    }.items())

In [4]:
def get_ephem(start, stop, step):
    resp = requests.get(get_url(start, stop, step))
    return resp.json()["result"].split("\n")

In [5]:
def find_origin(start, stop, step):
    ephem = get_ephem(start, stop, step)
    state = 0
    prev_dt = None
    dt = None
    prev_y = None
    y = None
    for line in ephem:
        match state:
            case 0:
                if "$$SOE" in line:
                    state = 1
            case 1:
                if "$$EOE" in line:
                    break
                elif "A.D." in line:
                    prev_dt = dt
                    dt = " ".join(line.split()[3:5])
                elif "X =" in line:
                    y = line.split("=")[2]
                    y = float(y[:y.find("Z")].strip())
                    if prev_y is not None and prev_y < 0.0 and y > 0.0:
                        break
                    prev_y = y
    return prev_dt, dt, prev_y, y

In [6]:
def year_origin(year):
    prev_dt, dt, _, _ = find_origin(f"{year}-01-01", f"{year + 1}-01-01", "1 day")
    prev_dt, dt, prev_y, y = find_origin(prev_dt, dt, "1 minute")
    if abs(prev_y) < abs(y):
        return prev_dt
    else:
        return dt

In [7]:
def dt_to_iso(dt):
    for k, v in MONTHS.items():
        dt = dt.replace(k, v)
    dt = dt.replace(" ", "T")
    return dt[:dt.find(".")]

In [8]:
def origin_ts(year):
    origin = year_origin(year)
    dt_str = dt_to_iso(origin)
    dt = datetime.fromisoformat(dt_str)
    ts = dt.timestamp()
    return ts

In [9]:
data = {}
for year in range(1970, 2021, 2):
    ts = origin_ts(year)
    print(year, ts)
    data[year] = ts

1970 22895580.0
1972 86011860.0
1974 149127480.0
1976 212244420.0
1978 275360400.0
1980 338476800.0
1982 401593020.0
1984 464709960.0
1986 527825760.0
1988 590941920.0
1990 654057780.0
1992 717174780.0
1994 780290940.0
1996 843407220.0
1998 906523200.0
2000 969640140.0
2002 1032755820.0
2004 1095872160.0
2006 1158988500.0
2008 1222105260.0
2010 1285220940.0
2012 1348337340.0
2014 1411453620.0
2016 1474570500.0
2018 1537686360.0
2020 1600802520.0


In [10]:
regr = LinearRegression()

In [11]:
X = np.array(sorted(data.keys())).reshape(-1, 1)

In [12]:
Y = np.array([data[x[0]] for x in X])

In [13]:
regr.fit(X, Y)

LinearRegression()

In [14]:
sidereal = 365.256363004 * 24.0 * 60.0 * 60.0

In [15]:
res = regr.predict(np.array([1970, 2020]).reshape(-1, 1))

In [16]:
phase = res[0] / sidereal
2.0 * np.pi - phase * 2.0 * np.pi, sidereal

(1.7247432929978155, 31558149.763545603)

In [23]:
X1 = np.array([2023.0, 2024.0, 2025.0, 2026.0]).reshape(-1, 1)
Y1 = np.array([1703215620.0, 1734772860.0, 1766329380.0, 1797886200.0])
regr1 = LinearRegression()
regr1.fit(X1, Y1)
res1 = regr1.predict(np.array([2023.0]).reshape(-1, 1))

In [24]:
solstice = res1[0]
origin = res[1]
axial_direction = (solstice - origin) % sidereal / sidereal * 2.0 * np.pi
axial_direction

1.5407643946374219